In [2]:
import json
import wptools
import re
from collections import defaultdict

def parse_occupations(text):
    """Extracts multiple occupations from complex formatted strings."""
    # Handle {{hlist|...}} and {{flatlist|...}}
    if '{{' in text:
        text = re.sub(r'\[\[([^\]]+)\]\]', r'\1', text)  # Simplify links
        text = re.sub(r'{{[^|]*\|', '', text)  # Remove the template up to the first '|'
        text = re.sub(r'}}', '', text)  # Remove closing braces
    # Handle <br> tags and strip unwanted characters
    text = re.sub(r'<br\s*/?>', '|', text)  # Replace <br> with |
    
    # Remove additional unwanted characters
    text = re.sub(r'\[\[|\]\]', '', text)  # Remove double brackets
    text = re.sub(r'\* ', '', text)  # Remove asterisks with spaces

    # Split by '|' and new lines as well as commas now
    occupations = re.split(r'\||\n|,', text)
    return [occupation.strip("*").strip().lower() for occupation in occupations if occupation.strip()]


def get_occupation_from_wikipedia(entity):
    """Use wptools to fetch and parse the occupation from Wikipedia without verbose output."""
    try:
        page = wptools.page(entity, silent=True).get_parse(show=False)
        infobox = page.data['infobox']
        if 'occupation' in infobox:
            return parse_occupations(infobox['occupation'])
        else:
            return ["Occupation not listed"]
    except Exception as e:
        return [f"Error retrieving page: {str(e)}"]

def extract_occupations(filepath):
    """Extract occupations for entities in a JSONL file and count occurrences."""
    occupation_count = defaultdict(int)
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            entity = data.get('topic', 'Unknown')
            occupations = get_occupation_from_wikipedia(entity)
            for occupation in occupations:
                occupation_count[occupation] += 1
    return occupation_count

In [ ]:
filepath = 'data/labeled/ChatGPT.jsonl'
occupation_counts = extract_occupations(filepath)
with open('occupation_counts.txt', 'w', encoding='utf-8') as f:
    for occupation, count in sorted(occupation_counts.items(), key=lambda item: item[1], reverse=True):
        f.write(f"{occupation}, Count: {count}\n")

In [17]:
len(occupation_counts)

80

In [4]:
occupations_set = set(occupation_counts.keys())
occupations_set.remove("Occupation not listed")
occupations_set.remove("Error retrieving page: argument of type 'NoneType' is not iterable")
print(occupations_set)

{'organist', 'narrator', 'pastor', 'musician', 'narration', 'author', 'arranger', 'educator', 'military officer', 'composer; soloist', 'model (person)', 'naval surgeon', 'film and television actor', 'singer fitness instructor', 'broadcasting', 'composer', 'data scientist', 'economist', 'voice acting in japan', 'journalist', 'politician', 'host', 'film director', 'guitarist', 'environmentalist', 'singer-songwriter', 'lawyer', 'radio broadcaster', 'screenwriter', 'athlete /coach (sports authority of india )', 'revolutionary', 'essayist', 'comedian', 'locksmith', 'writer', 'record producer', 'entertainer', 'dancer', 'stage', 'media executive', 'actress', 'parliamentarian', 'poet', 'businessman', 'model', 'actor', 'tv personality', 'songwriter', 'deejay (jamaican)', 'professor-emeritus of ecumenics and mission', 'mountaineer', 'radio host', 'travel writer', 'sportsperson', 'producer', 'film actress', 'philanthropist', 'businesswoman', 'voice actor', 'geographer', 'director', 'architect', '

In [50]:
occupations_set = {'organist', 'narrator', 'pastor', 'musician', 'narration', 'author', 'arranger', 'educator', 'military officer', 'soloist', 'model', 'naval surgeon', 'fitness instructor', 'broadcasting', 'composer', 'data scientist', 'economist', 'journalist', 'politician', 'host', 'film director', 'guitarist', 'environmentalist', 'songwriter', 'lawyer', 'radio broadcaster', 'screenwriter', 'athlete', 'coach', 'revolutionary', 'essayist', 'comedian', 'locksmith', 'writer', 'record producer', 'entertainer', 'dancer', 'stage', 'media executive', 'actress', 'parliamentarian', 'poet', 'businessman', 'model', 'actor', 'tv personality', 'songwriter', 'professor', 'mountaineer', 'radio host', 'travel writer', 'sportsperson', 'producer', 'film actress', 'philanthropist', 'businesswoman', 'voice actor', 'geographer', 'director', 'architect', 'teacher', 'television host', 'playwright', 'animal-rights activist', 'singer', 'translator', 'novelist', 'rapper', 'deejay', 'film producer', 'entrepreneur', 'stuntman', 'sportsman', 'columnist'}

In [51]:
len(occupations_set)

72

In [2]:
import os
import json
from collections import defaultdict, Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/yc833/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/yc833/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
def parse_infobox_line(line):
    """Extracts and concatenates occupations from the infobox line."""
    occupations = set()
    tokens = line.split('\t')
    occupation_parts = []
    for token in tokens:
        if 'occupation_' in token:
            # Extract the part of the occupation from the token
            part = token.split(':', 1)[1].strip()
            part = part.replace('[[', '').replace(']]', '').replace('*', '').strip()
            occupation_parts.append(part)

    # Join all parts into one string and add to the set of occupations
    if occupation_parts:
        full_occupation = " ".join(occupation_parts)
        occupations.add(full_occupation)

    return occupations


def remove_substrings(occupation_list):
    result = []
    for occupation in occupation_list:
        if not any(occupation in other for other in occupation_list if occupation != other):
            result.append(occupation)
    return result


def filter_articles_by_occupation(dataset_dir, occupations_set, char_threshold, num_samples):
    """Collects articles based on specified occupations up to a sample limit."""
    articles = defaultdict(list)
    subsets = ['train/train', 'valid/valid', 'test/test']
    title_files = [os.path.join(dataset_dir, f"{subset}.title") for subset in subsets]
    box_files = [os.path.join(dataset_dir, f"{subset}.box") for subset in subsets]
    sent_files = [os.path.join(dataset_dir, f"{subset}.sent") for subset in subsets]
    nb_files = [os.path.join(dataset_dir, f"{subset}.nb") for subset in subsets]

    for idx, (title_file, box_file, sent_file, nb_file) in enumerate(zip(title_files, box_files, sent_files, nb_files)):
        with open(title_file, 'r') as tfile, open(box_file, 'r') as bfile, open(sent_file, 'r') as sfile, open(nb_file, 'r') as nfile:
            title_lines = tfile.readlines()
            sent_lines = sfile.readlines()
            nb_lines = [int(line.strip()) for line in nfile.readlines()]
            start_index = 0
            for i, (bline, num_sentences) in enumerate(zip(bfile, nb_lines)):
                if all(len(articles[occ]) >= num_samples for occ in occupations_set):
                    break
                # print("bline:", bline)
                entry_occupations = parse_infobox_line(bline)
                # print("entry:", entry_occupations)
                matched_occupations = [o for o in occupations_set if any(o in occ.lower() for occ in entry_occupations)]
                # print("matched", matched_occupations)
                matched_occupations = remove_substrings(matched_occupations)
                # print("matched2", matched_occupations)
                if matched_occupations:
                    summary = ' '.join(sent_lines[start_index:start_index + num_sentences]).strip()
                    name = title_lines[i].strip()
                    if len(summary) > char_threshold:
                        for occ in matched_occupations:
                            if len(articles[occ]) < num_samples:
                                articles[occ].append({'name': name, 'summary': summary})
                                if len(articles[occ]) == num_samples:
                                    break
                start_index += num_sentences

    return articles


def save_data_and_report(articles, data_output_path, report_output_path, attribute_output_path):
    """Saves detailed data to JSON and generates a report on frequent words, also saved to JSON."""
    standard_stop_words = set(stopwords.words('english'))
    custom_stop_words = ["born", "known", "also", "became", "name", "one"]
    stop_words = standard_stop_words.union(set(custom_stop_words))
    data = {}
    report_data = {}
    attribute_data = {}

    for occupation, entries in articles.items():
        word_counter = Counter()
        for entry in entries:
            words = [word.lower() for word in word_tokenize(entry['summary']) if word.lower() not in stop_words and word.isalpha()]
            word_counter.update(words)
        
        most_common_words = word_counter.most_common(20)
        data[occupation] = {
            'count': len(entries),
            'entries': entries
        }
        report_data[occupation] = {
            'count': len(entries),
            'frequent_words': [{word: count} for word, count in most_common_words]
        }
        attribute_data[occupation] = {
            'attribute': [word for word, _ in most_common_words]
        }
    
    print(len(report_data))
    print(report_data.keys())

    with open(data_output_path, 'w', encoding='utf-8') as f_data:
        json.dump(data, f_data, indent=4)

    with open(report_output_path, 'w', encoding='utf-8') as f_report:
        json.dump(report_data, f_report, indent=4)

    with open(attribute_output_path, 'w', encoding='utf-8') as f_report:
        json.dump(attribute_data, f_report, indent=4)



In [55]:
dataset_dir = 'wikipedia-biography-dataset/wikipedia-biography-dataset'
char_threshold = 200
num_samples = 30
data_output_path = 'output/occupation_data.json'
report_output_path = 'output/occupation_report.json'
attribute_output_path = 'output/occupation_attribute.json'

articles_by_occupation = filter_articles_by_occupation(dataset_dir, occupations_set, char_threshold, num_samples)
save_data_and_report(articles_by_occupation, data_output_path, report_output_path, attribute_output_path)

71
dict_keys(['singer', 'actress', 'businesswoman', 'politician', 'composer', 'arranger', 'musician', 'film director', 'actor', 'playwright', 'poet', 'writer', 'essayist', 'songwriter', 'travel writer', 'director', 'teacher', 'lawyer', 'screenwriter', 'record producer', 'film producer', 'architect', 'model', 'comedian', 'author', 'novelist', 'producer', 'professor', 'journalist', 'translator', 'coach', 'host', 'organist', 'rapper', 'radio host', 'entrepreneur', 'athlete', 'philanthropist', 'businessman', 'economist', 'revolutionary', 'environmentalist', 'stage', 'tv personality', 'dancer', 'educator', 'narration', 'guitarist', 'entertainer', 'voice actor', 'columnist', 'narrator', 'television host', 'media executive', 'sportsman', 'broadcasting', 'film actress', 'military officer', 'pastor', 'stuntman', 'radio broadcaster', 'deejay', 'geographer', 'mountaineer', 'soloist', 'sportsperson', 'fitness instructor', 'data scientist', 'parliamentarian', 'locksmith', 'naval surgeon'])


1. Now getting the correct summaries. Still some problem with occupation. Seeing slashes "actor/screenwriter/executive", but no spaces
2. Now using occupation from occupation_set, but not parsing from info box correctly. 
3. Changed from set to string, and can now have occupations with space.
4. changed from 20 to 30
problem: writer is also included for song writer
    occupation_1:film
    occupation_2:director
    occupation_3:,
    occupation_4:film
    occupation_5:producer
5. fixed the above. Now if you have 'writer' and 'song writer' in matched, we only keep 'song writer'